# Feature Engineering

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data 

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

from sklearn.ensemble import RandomForestRegressor
# Feature Selection Library
from boruta import BorutaPy

## Read Data

In [2]:
full_data = get_simplified_data()

In [3]:
full_data.head()

,ID,Time,WF,T,CLCT,Set,Wind Speed 100m,Wind Direction 100m,Wind Speed 10m,Wind Direction 10m
0,1,01/05/2018 01:00,WF1,286.440000,82.543144,Train,3.958410,0.966685,1.287613,-0.226923
1,2,01/05/2018 02:00,WF1,286.260000,99.990844,Train,2.831607,0.535998,2.524975,-0.164453
2,3,01/05/2018 03:00,WF1,286.575000,98.367235,Train,1.249405,0.215257,1.731130,-0.956996
3,4,01/05/2018 04:00,WF1,285.842832,94.860604,Train,1.245551,1.226732,1.183180,-0.948583
4,5,01/05/2018 05:00,WF1,285.967452,95.905879,Train,1.266747,-1.491457,0.414344,-0.792171


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [6]:
WF = 'WF1'
data = full_data[full_data['WF']==WF]

## Feature Creation

### Wind Speed Vector

In [ ]:
feature_data = data[['ID','Time','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

Changing Reference for Negative Angle:

In [ ]:
feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m'] = 360 - feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m']
feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m'] = 360 - feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m']
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

## Tsfresh

First, we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

In [7]:
feature_data = pd.DataFrame()
for wf in full_data['WF'].unique():
    for variable in ['T', 'CLCT', 'Wind Speed 100m','Wind Direction 100m', 'Wind Speed 10m', 'Wind Direction 10m']:
        df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
        X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
        X['Feature'] = variable
        feature_data = feature_data.append(X)

Feature Extraction:  80%|████████  | 12/15 [01:43<00:19,  6.38s/it]

KeyboardInterrupt: 

## Feature Selection

Here we do the Feature Selection using Borutapy, a Python Implementation of the Famous R Method. For the method we use a Random Forest Regressor.

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

In [ ]:
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [ ]:
feat_selector.fit(X, y)

In [ ]:
feat_selector.ranking_